# EECE 5554 Lab 1: GNSS Drivers and Data

## Learning Objectives

Lab 1 introduces several concepts that are critical to sensing and navigation in robotics. You will use a GNSS receiver to collect latitude and longitude data, convert latitude and longitude to UTM zone and letter, and publish this information in a custom ROS message. You will also analyze walking and stationary data that you collect.

By the end of this lab, we expect that a well-prepared student will be able to: 
- Write a Python program to parse data read over USB serial 
- Convert latitude and longitude to UTM using the UTM package
- Define a custom ROS message 
- Publish a custom message using a ROS publisher node
- Save data using a .bag file 

## Information of Note

### Hardware: 
- GPS puck (provided by course)
- Laptop with sufficient battery life for at least two 10 minute data collections

### Software:
- Working Ubuntu from Lab 0
- Working ROS installation from Lab 0
- A serial emulator called Minicom: https://help.ubuntu.com/community/Minicom. Get it using <code> sudo apt install minicom </code>
- Turbo87's package for converting between lat/long and UTM: https://github.com/Turbo87/utm . Get it using <code> pip3 install utm</code>
- The course sensor emulator. Get it using <code> git clone https://github.com/ECE-RSN-Staff/sensor_emulator </code>
- The course Lab 1 autograder and examples. Get them using <code> git clone https://github.com/ECE-RSN-Staff/lab_1 </code>
- (Optional but recommended): A modern integrated development environment (IDE) like VS: https://code.visualstudio.com/docs/setup/linux

### The GNSS driver:
During this lab, you will write a driver that:
- Reads the string the GNSS puck outputs from the serial port
- Breaks this string into useful information (latitude, longitude, etc.)
- Converts the string to specified ROS formats for lat, lon, UTM easting, and UTM northing
- Merges the information into a custom message
- Publishes the message

### Individual and team work: 
Individual work: Write your own device driver for data acquisition and collect datasets. <br>
Team work: Share the GPS based GNSS puck after you have tested with the emulator

### Data sets: 
- One 5 minute .bag with stationary data in an open area (e.g., no trees or buildings within 10 m). Ideally, you will be close to some landmark where you can find the lat/long/alt coordinates.
- One 5 minute .bag with stationary data in an occluded area (e.g., trees or buildings nearby). Ideally, you will be close to some landmark where you can find the lat/long/alt coordinates.
- One .bag recording as you walk approximately 200 m in a straight line. Ideally, this dataset will include occluded and open areas, but anything is OK. Please make sure you make a note of this location and your start and end positions!

### Repository structure:

You will need to follow the naming convention laid out here to have an appropriate structure that can be checked by the Lab 1 autograder and to receive credit for your work. Please make sure you are following these conventions **exactly**-- spaces and capitalization matter!

Your Github repo should look like the table below. **Please note that your build and devel directories should not be pushed to Git!**


| EECE5554 | | | | | 
| :----- | :----- | :----- | :----- | :----- |
| | gnss/ | | | |
| | | gps_driver/ | | |
| | | | CMakeLists.txt | |
| | | | package.xml  | |
| | | | src/ | | 
| | | | | standalone_driver.py | 
| | | | msg/ | | 
| | | | | Customgps.msg | 
| | | | launch/ | | 
| | | | | standalone_driver.launch |
| | | analysis/ | | |
| | | | your analysis scripts | |
| | | data/ | | |
| | | | your .bag files | |

### What, where, and how to submit: 

Pust your driver and other files to your Github repo. Post your plots to the Lab 1 “quiz” listed on Canvas. No time limit, as many re-tries as you like, we’re just using the “quiz” structure because it gives easy file uploads + free response spaces.

## Onto the Lab!

<div class="alert alert-block alert-info">
<b>Linux Reminder:</b> If you want to understand any command in the Linux terminal, you can type the following in the terminal: <code> man command_name </code>
</div>

### Step 1: Write a Python script to read data from your GPS puck and convert to UTM

If you have experience writing Python scripts and/or ROS nodes, you can forge ahead. Otherwise, you might want to check out some of the other notebooks in Lab 1's directory for additional help with writing Python scripts, ROS publishers, ROS messages, and ROS bags. The Lab1_Pythonhelp notebook breaks down the steps below into manageable chunks related to Python, Lab1_ROShelp notebook breaks down the steps of a ROS publisher, and Lab1_SampleROSPub breaks down the components of an example ROS publisher node written in Python. Check them out!

Your complete driver should: 
- Read in and parse a GPGGA string into latitude, longitude, UTC, and HDOP
- Convert latitude and longitude into UTM values using the UTM package specified in "software" above
- Publish lat/lon/UTM data in a custom message named Customgps.msg with the following fields and data types:
    - header (type: Header)
        - header.frame_id should be 'GPS1_Frame' (type: string)
        - header.stamp.sec should be UTC **from the GPS sensor** converted to epoch time. Please do not use system time for UTC, but you will need to use system time to get epoch time to the current day. (type: uint32)
        - header.stamp.nsec should be remaining nanoseconds (type: uint32). You may have a value of 0 for this, depending on your GPS unit.
    - latitude (type: float64)
    - longitude (type: float64)
    - altitute (type: float64)
    - utm_easting (type: float64)
    - utm_northing (type: float64)
    - zone (type: uint8)
    - letter (type: string)
    - hdop (type: float64)
    - gpgga_read (type: string, this should be the actual string you read from the puck)
- Be launched from roslaunch using <code>roslaunch standalone_driver.launch port:=''any_port_name''</code>

If you want some help with those steps, please see the Lab1_Pythonhelp and Lab1_ROShelp notebooks.

Please remember that you'll need to update your CMakeLists.txt and package.xml to find your custom message. 

### Step 2: Set up your system to read data from the serial port

- Install Minicom and clone the sensor emulator repo as described in "software" above
- Configure Minicom's settings: 
    - <code>sudo chmod 777 /etc/minicom</code>
    - <code>minicom -s</code>
    - In Minicom's window, change the bps/par/bits to 4800 (the baud rate)
    - Change Hardware Flow Control to "no"
    - Save settings as default
    - Run minicom with <code>minicom</code> to see the strings the emulator is writing to the port
- Run the sensor emulator and verify you can see appropriate strings through Minicom. Please see the sensor emulator readme on Github if you're not sure how to run it, or you can run <code>python3 sensor_emulator -h</code>.


- Set up the GPS puck settings
    - Plug in your GPS puck and accept any popups if you are running on a VM
    - In terminal, list the devices with <code> ls –lt /dev/tty* | head</code>. You will see a list of devices from which you have to figure out your device file identifier. Let us say it is  <code>/dev/ttyUSB2</code>
    - Set the read write permissions for reading the device properly: <code>sudo chmod 777 /dev/ttyUSB2</code>
    
- Close Minicom before beginning testing with your driver

### Step 3: Write a launch file and make the driver accept any port

Steps 3-5 will be necessary for completing the assignment, but you can collect and run data analysis first (step 5) and finish this later. There is a very strong advantage to finishing these steps first, since you know you will be collecting good data!

Add a feature to run your driver with some argument that contains the path to the serial port of the GPS puck (example: /dev/ttyUSB2). This step allows us to connect it to any port without the script failing.

Even though this driver is now more modular, on a real robot we can have many sensors, so we want to write files that will launch multiple nodes at once. This is where we shall use the power of ROS.

Create a launch file called <code>standalone_driver.launch</code> in <code>~/catkin_ws/src/gps_driver/launch/</code>. This launch file should be able to take in an argument called “port” which we will specify for the puck’s port. Your driver should be able to be launched from the terminal with <code>roslaunch standalone_driver.launch port:="any_port_name"</code>. 

### Step 4: Push everything to your Github repo (but exclude build and devel directories!)

Once you've both committed and pushed, make sure you can see the files on your Github repo.

### Step 5: Use the Lab 1 autograder to check your work

If you get a success message, you're done, congrats! If something failed, follow up on the error. Definitely make sure your Github repo structure looks right.


### Step 6: Collect GNSS data

In separate rosbag recordings, you should collect three data sets: 
- 5 minutes of data in an open spot far away from buildings, trees, etc. Make sure you are in a known location, and use its coordinates on Google Earth as the “known” position.
- 5 minutes of data in an occluded spot near buildings or trees, but where you can still collect GPGGA strings. Make sure you are in a known location and use its coordinates on Google Earth as the “known” position.
- Data collected while walking in a straight line for 200-300 m. Make sure you note your start and end positions.

### Step 7: Analyze your collected data sets

Subtract the first point from each data set to remove the easting and northing offset. Then, find the centroid of each data set, the deviation in easting, and the deviation in northing. Want some more help? See the stats notebook and here: https://www.gnss.ca/app_notes/APN-029_GPS_Position_Accuracy_Measures_Application_Note.html

Make the following plots: 
- Stationary northing vs. easting scatterplots (open and occluded on same fig with different markers)
    - Plot after subtracting the centroid from each data point
    - Indicate the centroid value in easting and northing somewhere on the plot or text
- Stationary altitude vs. time plot (open and occluded on same fig with different markers) 
- Stationary histogram plots for position (open and occluded on different figs) 
    - Calculate the Euclidean distance from each point to your centroid and plot these values in the histogram 
- Moving data northing vs. easting scatterplot with line of best fit (open and occluded on same fig with different markers)
- Moving data altitude vs. time plot (open and occluded on same fig with different markers) 

Plots should ALWAYS include axis labels, units, and legends for more than one dataset. These values will be an important part of your score. You will further discuss your data analysis in Lab 2.

Please use the Canvas Lab1 quiz to upload your plots and answers to these questions.

### Lab Scoring

| Driver | 50 pct | Points | 
| :---- | :---- | :---- |
| | GPGGA string parsing | 10 | 
| | Converts lat/lon to decimal | 10 | 
| | Converts decimal to UTM | 10 | 
| | Correct time handling | 10 | 
| | Correct ROS message structure | 10 | 
| Data analysis | 50 pct | | 
| | Stationary northing vs. easting scatterplot (open and occluded on same fig) | 10 | 
| | Stationary altitude plot (open and occluded on same fig) | 10 | 
| | Stationary histogram plots (open and occluded on same fig) | 10 | 
| | Moving data scatterplot | 10 | 
| | Moving data altitude | 10 | 